<a href="https://colab.research.google.com/github/daspartho/learn-pytorch/blob/main/experiment_tracking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
import torchvision
import torch

from torch import nn
from torchvision import transforms

try:
    from torchinfo import summary
except:
    print("[INFO] Couldn't find torchinfo... installing it.")
    !pip install -q torchinfo
    from torchinfo import summary

try:
    from modular  import data_setup, engine
except:
    print("[INFO] Couldn't find modular scripts... downloading from GitHub.")
    !git clone https://github.com/daspartho/learn-pytorch
    !mv learn-pytorch/modular .
    !rm -rf learn-pytorch
    from modular import data_setup, engine

[INFO] Couldn't find torchinfo... installing it.
[INFO] Couldn't find modular scripts... downloading from GitHub.
Cloning into 'learn-pytorch'...
remote: Enumerating objects: 118, done.
remote: Counting objects: 100% (118/118), done.
remote: Compressing objects: 100% (114/114), done.
remote: Total 118 (delta 52), reused 8 (delta 1), pack-reused 0
Receiving objects: 100% (118/118), 4.91 MiB | 12.23 MiB/s, done.
Resolving deltas: 100% (52/52), done.


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [3]:
def set_seeds(seed: int=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

In [4]:
!python modular/get_data.py

Did not find data/pizza_steak_sushi directory, creating one...
Unzipping data...


In [5]:
train_dir = "data/pizza_steak_sushi/train"
test_dir = "data/pizza_steak_sushi/test"

weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
transforms = weights.transforms()

train_dataloader, test_dataloader = data_setup.create_dataloaders(
    train_dir,
    test_dir,
    transforms,
    batch_size=32,
)

In [6]:
model = torchvision.models.efficientnet_b0(weights=weights).to(device)

for param in model.features.parameters():
    param.requires_grad = False

set_seeds()

model.classifier = nn.Sequential(
    nn.Dropout(
        p=0.2, 
        inplace=True
        ),
    nn.Linear(
        in_features=1280, 
        out_features=3, 
        bias=True
        )
    )

summary(model=model,
        input_size = (32,3,224,224),
        col_names=["input_size", "output_size", "num_params", "trainable"],
        row_settings=["var_names"]
        )

Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-3dd342df.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-3dd342df.pth


  0%|          | 0.00/20.5M [00:00<?, ?B/s]

Layer (type (var_name))                                      Input Shape               Output Shape              Param #                   Trainable
EfficientNet (EfficientNet)                                  [32, 3, 224, 224]         [32, 3]                   --                        Partial
├─Sequential (features)                                      [32, 3, 224, 224]         [32, 1280, 7, 7]          --                        False
│    └─Conv2dNormActivation (0)                              [32, 3, 224, 224]         [32, 32, 112, 112]        --                        False
│    │    └─Conv2d (0)                                       [32, 3, 224, 224]         [32, 32, 112, 112]        (864)                     False
│    │    └─BatchNorm2d (1)                                  [32, 32, 112, 112]        [32, 32, 112, 112]        (64)                      False
│    │    └─SiLU (2)                                         [32, 32, 112, 112]        [32, 32, 112, 112]        --         

In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [8]:
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter()

In [9]:
from tqdm.auto import tqdm
from typing import Dict, List, Tuple

def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          loss_fn: torch.nn.Module,
          optimizer: torch.optim.Optimizer,
          epochs: int,
          device: torch.device) -> Dict[str, List]:
    
    results = {"train loss":[],
               "train acc":[],
               "test loss":[],
               "test acc":[]
               }
    
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = engine.train_step(model, 
                                           train_dataloader, 
                                           loss_fn, 
                                           optimizer,
                                           device)
        test_loss, test_acc = engine.test_step(model, 
                                        train_dataloader, 
                                        loss_fn,
                                        device)

        print(f"Epoch: {epoch} Train Loss: {train_loss:.3f} Train Acc: {train_acc:.3f} Test Loss: {test_loss:.3f} Test Acc: {test_acc:.3f}")
        
        results["train loss"].append(train_loss)
        results["train acc"].append(train_acc)
        results["test loss"].append(test_loss)
        results["test acc"].append(test_acc)

        writer.add_scalars(main_tag='Loss',
                           tag_scalar_dict={"train_loss": train_loss,
                                            "test_loss": test_loss},
                           global_step=epoch)
        writer.add_scalars(main_tag='Acc',
                           tag_scalar_dict={"train_acc": train_acc,
                                            "test_acc": test_acc},
                           global_step=epoch)
        writer.add_graph(model,
                         input_to_model=torch.randn(32,3,224,224).to(device))
    
    writer.close()

    return results

In [10]:
set_seeds()
result = train(model,
               train_dataloader,
               test_dataloader,
               loss_fn,
               optimizer,
               epochs=5,
               device=device)

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 0 Train Loss: 1.098 Train Acc: 0.383 Test Loss: 0.850 Test Acc: 0.770
Epoch: 1 Train Loss: 0.889 Train Acc: 0.668 Test Loss: 0.751 Test Acc: 0.883
Epoch: 2 Train Loss: 0.811 Train Acc: 0.676 Test Loss: 0.647 Test Acc: 0.867
Epoch: 3 Train Loss: 0.689 Train Acc: 0.875 Test Loss: 0.564 Test Acc: 0.922
Epoch: 4 Train Loss: 0.644 Train Acc: 0.777 Test Loss: 0.447 Test Acc: 0.930


In [12]:
result

{'train loss': [1.0977481752634048,
  0.8889638558030128,
  0.8105577006936073,
  0.6885548532009125,
  0.6444144323468208],
 'train acc': [0.3828125, 0.66796875, 0.67578125, 0.875, 0.77734375],
 'test loss': [0.8495295643806458,
  0.7506107911467552,
  0.646628312766552,
  0.5640857443213463,
  0.4469163082540035],
 'test acc': [0.76953125, 0.8828125, 0.8671875, 0.921875, 0.9296875]}